### Just trying to organize one big tall dataset here, each text (speech or article) as its own row

In [35]:
import os
import re
import unicodedata
import numpy as np
import pandas as pd
import warnings
import nltk
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer

from nltk.tokenize import TreebankWordTokenizer
from nltk.tokenize import sent_tokenize
nltk.download('punkt')
word_token = TreebankWordTokenizer()

[nltk_data] Downloading package punkt to /home/muddy/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [12]:
# Load up the files
paths = ['./speeches/', './NYTimes/', './WSJ/'] 
list_of_files = []

dates = pd.read_csv('dateSpeeches.csv')
for path in paths:
    for root, dirs, files in os.walk(path):
        for file in files:
            if file.endswith('.txt'):
                list_of_files.append(os.path.join(root,file))

speeches = []
for file in list_of_files:
    with open(file, encoding='utf-8') as f:
        #print(file)
        text = f.read()
    f.close()
    speeches.append([text, file])

#clean out goofy unicode  space characters 
speeches = [(unicodedata.normalize("NFKD", speech[0]), speech[1]) for speech in speeches if len(speech)>0 ]

# remove [stuff] in between square brackets
def remove_bracket(text):
    return re.sub('(\[[^w]*\]\s)', '',text)
speeches = [(remove_bracket(speech[0]), speech[1]) for speech in speeches]

<A HREF="https://docs.python.org/3/library/datetime.html#strftime-and-strptime-format-codes">Python datetime format codes</A>

In [16]:
pos = pd.read_csv('POSFacetPlotData.csv', index_col=None) #date, source, pos, proportion
wide = pd.read_csv('df_encodings.csv', index_col=None) #file, text, emos, enc vectors
pca = pd.read_csv('pcaBiplotData.csv', index_col=None) #date, source, enc1-300, PCA1, PCA2
emo = pd.read_csv('facetPlotData.csv', index_col=None) #date, source, emo, emoValue
pos['date'] = pd.to_datetime(pos['date'], format='%Y-%m-%d')
wide['date'] = pd.to_datetime(wide['date'], format='%Y-%m-%d')
pca['date'] = pd.to_datetime(pca['date'], format='%Y-%m-%d')
emo['date'] = pd.to_datetime(emo['date'], format='%Y-%m-%d')

In [17]:
pos.head(3)

,date,source,pos,proportion
0,2008-06-04,oba,PRON,0.064597
1,2008-06-04,oba,NUM,0.058727
2,2008-06-04,oba,DET,0.064638


In [18]:
pos_wide=pos.pivot(index=['date', 'source'],columns='pos', values='proportion').reset_index()
pos_wide.shape

(300, 18)

In [19]:
pos_wide.head()

pos,date,source,ADJ,ADP,ADV,AUX,CCONJ,DET,INTJ,NOUN,NUM,PART,PRON,PROPN,PUNCT,SCONJ,SYM,VERB
0,2008-06-04,nyt,0.064458,0.065088,0.062400,0.064962,0.064416,0.063408,0.055598,0.064920,0.065046,0.065004,0.065130,0.064668,0.065172,0.061602,0.043420,0.064710
1,2008-06-04,oba,0.064649,0.064670,0.064440,0.064398,0.064471,0.064638,0.054967,0.064503,0.058727,0.064315,0.064597,0.064680,0.064691,0.064377,0.000000,0.064628
2,2008-06-04,wsj,0.072536,0.071300,0.070185,0.071579,0.072456,0.072177,0.000000,0.072616,0.065402,0.072098,0.071938,0.072217,0.072656,0.070344,0.000000,0.072496
3,2008-11-05,nyt,0.065299,0.065636,0.065602,0.064593,0.065131,0.065266,0.042603,0.065972,0.065064,0.065703,0.065837,0.065669,0.066005,0.063315,0.042569,0.065736
4,2008-11-05,oba,0.067288,0.067345,0.064371,0.067174,0.067145,0.067260,0.066802,0.066745,0.065544,0.065000,0.067088,0.067374,0.067403,0.066230,0.000000,0.067231


In [20]:
wide.head(1)

,file,text_oba,text_wsj,text_nyt,emo_oba,emo_nyt,emo_wsj,date,obafear,obaanger,...,NationalReview,FoxNews,Intercept,enc_oba,enc_nyt,enc_wsj,enc_fn,enc_iw,enc_nr,enc_int
0,2008-06-04-ObamaNomination.txt,thank you thank you what a what a wonderful re...,sen barack obama captured enough convention de...,senator barack obama claimed the democratic pr...,<nrclex.NRCLex object at 0x7f09f2a0eda0>,<nrclex.NRCLex object at 0x7f09e5416260>,<nrclex.NRCLex object at 0x7f09e2782260>,2008-06-04,0.095798,0.055462,...,NaN,NaN,NaN,"[-1.5006276, 1.9996313, -3.2598364, -0.2249771...","[-1.8957838, 1.5588127, -2.2415352, 0.4132859,...","[-2.0421023, 1.3786627, -2.2437146, 0.49334085...",NaN,NaN,NaN,NaN


In [21]:
enc_oba=pd.melt(wide, id_vars =['date'], value_vars =['enc_oba'],var_name='source', value_name='enc_value')
enc_oba['source']='oba'
enc_nyt=pd.melt(wide, id_vars =['date'], value_vars =['enc_nyt'],var_name='source', value_name='enc_value')
enc_nyt['source']='nyt'
enc_wsj=pd.melt(wide, id_vars =['date'], value_vars =['enc_wsj'],var_name='source', value_name='enc_value')
enc_wsj['source']='wsj'
encodings=pd.concat([enc_oba,enc_nyt,enc_wsj])

In [22]:
encodings.shape

(300, 3)

In [23]:
pca_values=pca[['date', 'source', 'PCA1', 'PCA2']].copy()
pca_values['source']=pca_values['source'].replace('text_', '', regex=True)
pca_values.head(3)

,date,source,PCA1,PCA2
0,2008-06-04,oba,5.803243,1.420757
1,2008-06-04,nyt,-3.245703,3.328605
2,2008-06-04,wsj,-3.515471,2.659997


In [24]:
emo.head(13)

,date,source,emotion,emoValue
0,2008-06-04,oba,fear,0.095798
1,2008-06-04,oba,anger,0.055462
2,2008-06-04,oba,trust,0.159664
3,2008-06-04,oba,surprise,0.048739
4,2008-06-04,oba,positive,0.233613
5,2008-06-04,oba,negative,0.115966
6,2008-06-04,oba,sadness,0.042017
7,2008-06-04,oba,disgust,0.023529
8,2008-06-04,oba,joy,0.095798
9,2008-06-04,oba,anticipation,0.129412


<A HREF="https://pandas.pydata.org/Pandas_Cheat_Sheet.pdf">Pandas Cheat Sheet</A>

In [25]:
emo_wide=emo.pivot(index=['date', 'source'],columns='emotion', values='emoValue').reset_index()
emo_wide.shape

(300, 12)

In [26]:
emo_wide.head(3)

emotion,date,source,anger,anticipation,disgust,fear,joy,negative,positive,sadness,surprise,trust
0,2008-06-04,nyt,0.058824,0.121849,0.012605,0.063025,0.079832,0.130252,0.281513,0.075630,0.046218,0.130252
1,2008-06-04,oba,0.055462,0.129412,0.023529,0.095798,0.095798,0.115966,0.233613,0.042017,0.048739,0.159664
2,2008-06-04,wsj,0.036630,0.087912,0.025641,0.051282,0.106227,0.095238,0.326007,0.054945,0.047619,0.168498


In [27]:
#pos_wide,encodings,pca_values,emo_wide  pd.merge(adf, bdf, how='left', on='x1')
df1 = pd.merge(pos_wide, encodings, how='left', on=['date', 'source'])
df2 = pd.merge(df1, pca_values, how='left', on=['date', 'source'])
df3 = pd.merge(df2, emo_wide, how='left', on=['date', 'source'])

In [28]:
df3.corr()[df3.corr()>0.5].loc[['PCA1', 'PCA2'] ].transpose()

/tmp/ipykernel_8029/3981209273.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  df3.corr()[df3.corr()>0.5].loc[['PCA1', 'PCA2'] ].transpose()


,PCA1,PCA2
ADJ,NaN,NaN
ADP,NaN,NaN
ADV,NaN,NaN
AUX,NaN,NaN
CCONJ,NaN,NaN
DET,NaN,NaN
INTJ,NaN,NaN
NOUN,NaN,NaN
NUM,NaN,NaN
PART,NaN,NaN


In [29]:
text_oba=pd.melt(wide, id_vars =['date'], value_vars =['text_oba'],var_name='source', value_name='text')
text_oba['source']='oba'
text_nyt=pd.melt(wide, id_vars =['date'], value_vars =['text_nyt'],var_name='source', value_name='text')
text_nyt['source']='nyt'
text_wsj=pd.melt(wide, id_vars =['date'], value_vars =['text_wsj'],var_name='source', value_name='text')
text_wsj['source']='wsj'
texts=pd.concat([text_oba,text_nyt,text_wsj])

In [30]:
texts

,date,source,text
0,2008-06-04,oba,thank you thank you what a what a wonderful re...
1,2008-11-05,oba,hello chicago if there is anyone out there who...
2,2009-01-05,oba,all right well thanks for taking the time to c...
3,2009-01-20,oba,i barack hussein obama do solemnly swear that ...
4,2009-04-02,oba,good afternoon or good evening we're running a...
...,...,...,...
95,2016-07-28,wsj,president barack obama with the end of his own...
96,2016-08-17,wsj,days after floodwaters ravaged southeastern lo...
97,2016-09-05,wsj,the group of 20 nations is close to an agreeme...
98,2016-11-09,wsj,republican donald trump won the 2016 election ...


<A HREF="https://practicaldatascience.co.uk/data-science/how-to-use-python-regular-expressions-to-extract-information">Decent regex guide</A><BR><A HREF="https://regexr.com/">RegExr</A>

In [32]:
def get_source(text):
    regex = "[^./][a-zA-Z]+[^/]"
    string = re.findall(regex, str(text))[0]
    if string == 'speeches': string = 'oba'
    if string == 'NYTimes': string = 'nyt'
    return string.lower()

def get_date(text):
    regex = "([0-9]+[\-][0-9]+[\-][0-9]+)"
    return re.findall(regex, str(text))[0]

def get_filename(text):
    regex = "[-]([a-zA-Z]+)"
    return re.findall(regex, str(text))[0]

speech_df = pd.DataFrame(speeches)
print(get_source(speeches[50][1]))

print(get_date(speeches[50][1]))

print(get_filename(speeches[50][1]))

oba
2008-06-04
ObamaNomination


In [33]:
cols = ['text', 'filepath']
text_df = pd.DataFrame(speeches, columns=cols)
text_df['date'] = text_df['filepath'].apply(get_date)
text_df['date'] = pd.to_datetime(text_df['date'], format='%Y-%m-%d')
text_df['source'] = text_df['filepath'].apply(get_source)
text_df

,text,filepath,date,source
0,"Good afternoon, everybody. One year ago this m...",./speeches/2014-07-01-Immigration.txt,2014-07-01,oba
1,"Good morning, everybody. I want to take just a...",./speeches/2009-12-25-UnderwearBomber.txt,2009-12-25,oba
2,"Hello, Chicago.\n\nIf there is anyone out ther...",./speeches/2008-11-05-ObamaElected.txt,2008-11-05,oba
3,"Good afternoon, everybody. As a candidate for ...",./speeches/2011-12-16-EndIraqWar.txt,2011-12-16,oba
4,"Thank you. Good evening, everybody. Good eveni...",./speeches/2012-04-29-WHCorrespondentsDinner.txt,2012-04-29,oba
...,...,...,...,...
298,"President Barack Obama, emphasizing U.S. allie...",./WSJ/2014-03-26-NATOBelgium.txt,2014-03-26,wsj
299,The White House cleared the way for tough new ...,./WSJ/2012-03-31-OilIran.txt,2012-03-31,wsj
300,"President Barack Obama, seeking to restore con...",./WSJ/2014-01-29-StateoftheUnion.txt,2014-01-29,wsj
301,President Barack Obama won re-election Tuesday...,./WSJ/2012-11-07-ObamaWinsRe-election.txt,2012-11-07,wsj


In [36]:
text_df['sentences'] = text_df['text'].apply(sent_tokenize)
text_df['words'] = text_df['text'].apply(word_token.tokenize)
text_df['num_sents'] = text_df['sentences'].apply(len)
text_df['num_words'] = text_df['words'].apply(len)
text_df['word_set'] = text_df['words'].apply(set)
text_df['num_unique_words'] = text_df['word_set'].apply(len)

In [43]:
#text_df.to_csv('text_sentences_words.csv', index=False)

In [38]:
text_df_numbers = text_df[['date', 'source', 'num_sents', 'num_words', 'num_unique_words']].copy()

In [39]:
df4 = pd.merge(df3, text_df_numbers, how='left', on=['date', 'source'])

In [40]:
df4

,date,source,ADJ,ADP,ADV,AUX,CCONJ,DET,INTJ,NOUN,...,fear,joy,negative,positive,sadness,surprise,trust,num_sents,num_words,num_unique_words
0,2008-06-04,nyt,0.064458,0.065088,0.062400,0.064962,0.064416,0.063408,0.055598,0.064920,...,0.063025,0.079832,0.130252,0.281513,0.075630,0.046218,0.130252,47,1459,620
1,2008-06-04,oba,0.064649,0.064670,0.064440,0.064398,0.064471,0.064638,0.054967,0.064503,...,0.095798,0.095798,0.115966,0.233613,0.042017,0.048739,0.159664,217,5850,936
2,2008-06-04,wsj,0.072536,0.071300,0.070185,0.071579,0.072456,0.072177,0.000000,0.072616,...,0.051282,0.106227,0.095238,0.326007,0.054945,0.047619,0.168498,64,1709,657
3,2008-11-05,nyt,0.065299,0.065636,0.065602,0.064593,0.065131,0.065266,0.042603,0.065972,...,0.070064,0.085987,0.136943,0.248408,0.079618,0.041401,0.114650,54,1862,735
4,2008-11-05,oba,0.067288,0.067345,0.064371,0.067174,0.067145,0.067260,0.066802,0.066745,...,0.060000,0.117778,0.115556,0.233333,0.062222,0.046667,0.160000,96,2254,755
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,2016-11-09,oba,0.069479,0.069178,0.069730,0.069078,0.068376,0.069429,0.042841,0.069329,...,0.034014,0.125850,0.074830,0.241497,0.034014,0.091837,0.187075,79,1303,470
296,2016-11-09,wsj,0.069166,0.068667,0.068939,0.068576,0.068078,0.069120,0.000000,0.069211,...,0.050909,0.069091,0.156364,0.221818,0.076364,0.080000,0.152727,61,1446,623
297,2017-01-11,nyt,0.068761,0.069864,0.067920,0.065872,0.069811,0.069916,0.041708,0.070074,...,0.073826,0.077181,0.127517,0.238255,0.073826,0.057047,0.161074,53,1275,558
298,2017-01-11,oba,0.066897,0.066921,0.066529,0.066823,0.066811,0.066885,0.066676,0.066493,...,0.091806,0.103187,0.115326,0.242792,0.050076,0.044765,0.157815,231,5138,1588


In [41]:
#df4.to_csv('tidy_data.csv', index=False)

In [42]:
df4.corr()[df4.corr()>0.5].loc[['PCA1', 'PCA2'] ].transpose()

/tmp/ipykernel_8029/221583640.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  df4.corr()[df4.corr()>0.5].loc[['PCA1', 'PCA2'] ].transpose()


,PCA1,PCA2
ADJ,NaN,NaN
ADP,NaN,NaN
ADV,NaN,NaN
AUX,NaN,NaN
CCONJ,NaN,NaN
DET,NaN,NaN
INTJ,NaN,NaN
NOUN,NaN,NaN
NUM,NaN,NaN
PART,NaN,NaN
